In [1]:
path = r'C:\Madhan\Analytics\Machine_Learning_project_work\AI_ML\bluepencil\model_improvement'

In [2]:
import pandas as pd
import nltk
import numpy as np
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt   
import matplotlib.style
plt.style.use('classic')
from nltk import word_tokenize
from nltk import download
from nltk.corpus import stopwords
#importing seaborn for statistical plots
import seaborn as sns
import sklearn.metrics as metrics
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import spacy


In [3]:
legal_data = pd.read_excel(path+'\legal_data_1.0.xlsx',sheet_name='ResultLegal')

In [4]:
legal_data.head()

,Index,Filename,FileContent,Label
0,0,1003143 D4183 GISS 2 pager for Life Sciences_V...,Creating trust in \r\nthe digital world\r\nEYâ...,0
1,1,1003143 D4183 GISS 2 pager for Life Sciences_V...,"It \r\ncaptures the responses of 1,755 partici...",0
2,2,1003143 D4183 GISS 2 pager for Life Sciences_V...,The following findings from the 36 participant...,0
3,3,1003143 D4183 GISS 2 pager for Life Sciences_V...,But they also indicate the need for considerab...,0
4,4,1003143 D4183 GISS 2 pager for Life Sciences_V...,Operating in a digital world invites \r\nnew ...,0


In [5]:
LGL_words = pd.read_excel(path+'\legal_data_1.0.xlsx',sheet_name='Sheet1')

In [6]:
legal_data.head()

,Index,Filename,FileContent,Label
0,0,1003143 D4183 GISS 2 pager for Life Sciences_V...,Creating trust in \r\nthe digital world\r\nEYâ...,0
1,1,1003143 D4183 GISS 2 pager for Life Sciences_V...,"It \r\ncaptures the responses of 1,755 partici...",0
2,2,1003143 D4183 GISS 2 pager for Life Sciences_V...,The following findings from the 36 participant...,0
3,3,1003143 D4183 GISS 2 pager for Life Sciences_V...,But they also indicate the need for considerab...,0
4,4,1003143 D4183 GISS 2 pager for Life Sciences_V...,Operating in a digital world invites \r\nnew ...,0


In [7]:
#legal_data.legal_word.value_counts()

In [8]:
legal_data.Label.value_counts()

0    30046
1      755
Name: Label, dtype: int64

In [9]:
X = legal_data.FileContent

In [10]:
print(X.iloc[0])

Creating trust in 
the digital world
EYâ€™s Global Information Security 
Survey 2015

Life sciences sector results

EYâ€™s Global Information Security Survey investigates the most important cybersecurity issues facing businesses today.


In [11]:
y = legal_data.Label

In [12]:
default_stopwords = set(nltk.corpus.stopwords.words('english'))

In [13]:
from nltk.corpus import wordnet
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    #print(tag)
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    output = tag_dict.get(tag[0]) 
    if output:
        return output
    else:
        return 'v'

In [14]:
def review_to_words( SD ):
    lm = WordNetLemmatizer()
    letters_only = re.sub("[^a-zA-Z]", " ", SD)
    #print(letters_only)
    sentences = nltk.word_tokenize(str(letters_only))
    #print(sentences)
    sentences = [word.lower() for word in sentences]
    no_stops = [word for word in sentences if not word in default_stopwords]
    #print(no_stops)
    lem_word = [lm.lemmatize(word,get_wordnet_pos(word)) for word in no_stops]
    #print(lem_word)
    #tags = st.tag(lem_word)
    #tagged_words = [word for word,tag in tags if tag!='PERSON' and tag!='LOCATION']
    tokens = [word.lower() for word in lem_word if len(word) > 2]
    #print(tokens)
    return( " ".join(tokens))

In [15]:
X.iloc[1]

'It \r\ncaptures the responses of 1,755 participants around the globe and across industries, and we base our findings and conclusions \r\non those insights and on our extensive global experience working with clients to help improve their cybersecurity.'

In [16]:
review_to_words(X.iloc[1])

'capture response participant around globe across industry base finding conclusion insight extensive global experience work client help improve cybersecurity'

In [17]:
print("Number of Rows Parsed......................\n")
corpus = []
num_SD = len(X)
%time
for i in range( 0, num_SD ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%5000 == 0 ):
        print("Cleaned Short Descriptions %d of %d" % ( i+1, num_SD ))
    corpus.append( review_to_words(str(X.iloc[i])))

Number of Rows Parsed......................

Wall time: 0 ns
Cleaned Short Descriptions 5000 of 30801
Cleaned Short Descriptions 10000 of 30801
Cleaned Short Descriptions 15000 of 30801
Cleaned Short Descriptions 20000 of 30801
Cleaned Short Descriptions 25000 of 30801
Cleaned Short Descriptions 30000 of 30801


In [30]:
X = pd.DataFrame(corpus,columns=['text'])

In [31]:
X.iloc[4]

text    operating digital world invite new challenge t...
Name: 4, dtype: object

In [32]:
X.text[30800]

'doubt tax certainty project well work action discuss would useful regard also'

In [33]:
X['leg'] = ''
leg = []
for idx, row in X.iterrows():
    line_txt = X.text[idx]
    for row1 in LGL_words.LGL:
        if row1 in line_txt:
            X['leg'][idx] = 'yes'

In [34]:
X.head()

,text,leg
0,create trust digital world global information ...,
1,capture response participant around globe acro...,
2,follow finding participant life science sector...,
3,also indicate need considerable improvement wo...,
4,operating digital world invite new challenge t...,yes


In [35]:
X.leg.unique()

array(['', 'yes'], dtype=object)

In [36]:
X.leg.value_counts()

       25679
yes     5122
Name: leg, dtype: int64

In [37]:
X.leg.replace(['','yes'],[0,1],inplace=True)

In [38]:
X.leg.head()

0    0
1    0
2    0
3    0
4    1
Name: leg, dtype: int64

In [39]:
y.value_counts()

0    30046
1      755
Name: Label, dtype: int64

In [40]:
X_train_txt, X_test_txt, y_train, y_test,leg_train,leg_test = train_test_split(X.text, y,X.leg,test_size=0.2, random_state=1)

In [41]:
print(X_train_txt.shape)
print(X_test_txt.shape)
print(y_train.shape)
print(y_test.shape)
print(leg_train.shape)
print(leg_test.shape)

(24640,)
(6161,)
(24640,)
(6161,)
(24640,)
(6161,)


In [12]:
tfidf_vector = TfidfVectorizer(stop_words='english', ngram_range=(1,4),max_features=3000,lowercase=True)
#X_dtm_train = tfidf_vec.fit_transform((X_train_txt))
#X_dtm_train.shape

In [43]:
#X_dtm_test = tfidf_vec.transform(X_test_txt)
#X_dtm_test.shape

In [44]:
df_corpus = pd.DataFrame(corpus,columns=['text'])

In [45]:
df_corpus.head()

,text
0,create trust digital world global information ...
1,capture response participant around globe acro...
2,follow finding participant life science sector...
3,also indicate need considerable improvement wo...
4,operating digital world invite new challenge t...


In [46]:
df_cleaned = pd.concat([df_corpus,legal_data['Label'],X['leg']],axis=1)

In [ ]:
df_cleaned.to_csv('training_data1.csv')

In [6]:
trainingset1 = pd.read_csv(path+'\\training_data1.csv')

In [7]:
trainingset1.Label.value_counts()

0    29651
1      755
Name: Label, dtype: int64

In [8]:
X_res = trainingset1.text
y_res = trainingset1.Label
feat_res = trainingset1.leg

In [9]:
X_train_txt, X_test_txt, y_train, y_test,feat_train,feat_test = train_test_split(trainingset1.text,trainingset1.Label,
                                                                                   trainingset1.leg,test_size=0.2, random_state=1)

In [13]:
X_train_dtm = tfidf_vector.fit_transform(X_train_txt.values.astype('U'))

In [14]:
feat_train = np.array(feat_train)

In [15]:
dense = X_train_dtm.todense()
feat_train = feat_train.reshape(-1,1)
final_train_dtm = np.append(dense,feat_train,1)

In [16]:
final_train_dtm.shape
y_train.shape

(24324,)

In [344]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN

In [345]:
ros = RandomOverSampler(random_state=10)

In [346]:
sm = SMOTE(random_state=42)

In [347]:
ada = ADASYN(random_state=11,)

In [348]:
X_res_train_new, y_res_train_new = ros.fit_resample(final_train_dtm, y_train)
#X_res_new, y_res_new = sm.fit_resample(final_train_dtm, y_res)
#X_res_new, y_res_new = ada.fit_resample(final_train_dtm, y_res)

In [349]:
print(X_res_train_new.shape)
print(y_res_train_new.shape)

(48058, 5001)
(48058,)


In [350]:
X_test_dtm = tfidf_vector.transform(X_test_txt.values.astype('U'))

In [351]:
feat_test = np.array(feat_test)

In [352]:
dense = X_test_dtm.todense()
feat_test = feat_test.reshape(-1,1)
final_test_dtm = np.append(dense,feat_test,1)

In [353]:
X_res_test_new, y_res_test_new = ros.fit_resample(final_test_dtm, y_test)


In [354]:
X_res_test_new.shape

(12034, 5001)

In [355]:
y_res_test_new.shape

(12034,)

In [356]:
rf_classifer = RandomForestClassifier(n_estimators=200,criterion='entropy',random_state=0,class_weight={0:.3,1:.7},max_depth=200)
rf_classifer.fit(X_res_train_new ,y_res_train_new)
y_pred_randforest = rf_classifer.predict(X_res_test_new)

In [357]:

print("RandForest ACCURACY:",metrics.accuracy_score(y_res_test_new,y_pred_randforest))
print('RandForest CLASSIFIER CONFUSTION MATRIX:',pd.crosstab(y_res_test_new,y_pred_randforest))
print("===> **RandForest F1 SCORE",metrics.f1_score(y_res_test_new, y_pred_randforest,average=None))
print("===> **RandForest Recall SCORE",metrics.recall_score(y_res_test_new, y_pred_randforest,average=None))
print("===> **RandForest Precision SCORE",metrics.precision_score(y_res_test_new, y_pred_randforest,average=None))
print("===> **RandForest ROC_AUC SCORE",metrics.roc_auc_score(y_res_test_new, y_pred_randforest,average=None))

RandForest ACCURACY: 0.5461193285690543
RandForest CLASSIFIER CONFUSTION MATRIX: col_0     0    1
row_0           
0      5940   77
1      5385  632
===> **RandForest F1 SCORE [0.68504209 0.18792745]
===> **RandForest Recall SCORE [0.98720293 0.10503573]
===> **RandForest Precision SCORE [0.52450331 0.89139633]
===> **RandForest ROC_AUC SCORE 0.5461193285690544


## Predicting for TrainingSet2

In [47]:
trainingset2 = pd.read_excel(path+'\\Trainingset2.xlsx',sheet_name='Sheet1')

In [48]:
trainingset2.columns

Index(['Index', 'filename', 'text', 'leg'], dtype='object')

In [49]:
trainingset2.leg.value_counts()

0    38654
1     5575
Name: leg, dtype: int64

In [50]:
X = trainingset2.text

In [51]:
print("Number of Rows Parsed......................\n")
corpus = []
num_SD = len(X)
%time
for i in range( 0, num_SD ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%5000 == 0 ):
        print("Cleaned Short Descriptions %d of %d" % ( i+1, num_SD ))
    corpus.append( review_to_words(str(X.iloc[i])))

Number of Rows Parsed......................

Wall time: 0 ns
Cleaned Short Descriptions 5000 of 44229
Cleaned Short Descriptions 10000 of 44229
Cleaned Short Descriptions 15000 of 44229
Cleaned Short Descriptions 20000 of 44229
Cleaned Short Descriptions 25000 of 44229
Cleaned Short Descriptions 30000 of 44229
Cleaned Short Descriptions 35000 of 44229
Cleaned Short Descriptions 40000 of 44229


In [52]:
len(corpus)

44229

In [53]:
df_corpus = pd.DataFrame(corpus,columns=['text'])

In [54]:
df_corpus.head()

,text
0,please inform principle reciprocity rule requi...
1,june follow year september follow year june fo...
2,particular asian company must residence seat f...
3,claim must also comply certain specific admini...
4,country also require company country residence...


In [55]:
df_cleaned = pd.concat([df_corpus,trainingset2['leg']],axis=1)

In [56]:
X_test_txt = tfidf_vector.transform(df_cleaned.text)

NotFittedError: TfidfVectorizer - Vocabulary wasn't fitted.

In [384]:
dense_test = X_test_txt.todense()

In [385]:
state_test = np.array(trainingset2.leg)
state_test = state_test.reshape(-1,1)
final_test_dtm = np.append(dense_test,state_test,1)

In [387]:
final_test_dtm.shape

(44229, 3001)

In [388]:
import pickle
test_path = r'C:\Madhan\Analytics\Machine_Learning_project_work\AI_ML\bluepencil\model_improvement'

In [389]:
rf_1 = pickle.load(open(test_path+'\RF_1.sav', 'rb'))
rf_2 = pickle.load(open(test_path+'\RF_2.sav', 'rb'))
rf_3 = pickle.load(open(test_path+'\RF_3.sav', 'rb'))
rf_4 = pickle.load(open(test_path+'\RF_4.sav', 'rb'))
rf_5 = pickle.load(open(test_path+'\RF_5.sav', 'rb'))
rf_6 = pickle.load(open(test_path+'\RF_6.sav', 'rb'))
rf_7 = pickle.load(open(test_path+'\RF_7.sav', 'rb'))
rf_8 = pickle.load(open(test_path+'\RF_8.sav', 'rb'))
rf_9 = pickle.load(open(test_path+'\RF_9.sav', 'rb'))
rf_10 = pickle.load(open(test_path+'\RF_10.sav', 'rb'))

In [390]:
y_pred_model1 = rf_1.predict_proba(final_test_dtm)
y_pred_model2 = rf_2.predict_proba(final_test_dtm)
y_pred_model3 = rf_3.predict_proba(final_test_dtm)
y_pred_model4 = rf_4.predict_proba(final_test_dtm)
y_pred_model5 = rf_5.predict_proba(final_test_dtm)
y_pred_model6 = rf_6.predict_proba(final_test_dtm)
y_pred_model7 = rf_7.predict_proba(final_test_dtm)
y_pred_model8 = rf_8.predict_proba(final_test_dtm)
y_pred_model9 = rf_9.predict_proba(final_test_dtm)
y_pred_model10 = rf_10.predict_proba(final_test_dtm)



In [391]:

n=np.vstack((y_pred_model1[:,0],y_pred_model2[:,0],y_pred_model3[:,0],y_pred_model4[:,0],y_pred_model5[:,0],
             y_pred_model6[:,0],y_pred_model7[:,0],
            y_pred_model8[:,0],y_pred_model9[:,0],y_pred_model10[:,0]))

In [392]:
n.shape

(10, 44229)

In [393]:
n[0:,]

array([[0.92391304, 0.31      , 0.39      , ..., 0.94      , 0.89      ,
        0.98517241],
       [0.31      , 0.3       , 0.39      , ..., 0.9       , 0.87      ,
        0.96      ],
       [0.78      , 0.25      , 0.24      , ..., 0.98      , 0.98      ,
        0.94      ],
       ...,
       [0.77      , 0.51      , 0.4575    , ..., 0.97      , 0.75      ,
        0.78      ],
       [0.9       , 0.37      , 0.33      , ..., 0.92      , 1.        ,
        0.87      ],
       [0.91      , 0.38      , 0.36      , ..., 0.98      , 0.94      ,
        0.98132203]])

In [394]:
n = n.transpose()
final_df = n

In [395]:
rf_2nd = pickle.load(open(test_path+'\RF_2nd_layer.sav', 'rb'))
final_pred = rf_2nd.predict(final_df)
final_pred

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

In [396]:
final_pred = pd.DataFrame(final_pred,columns=['pred'])

In [398]:
trainingset2.shape

(44229, 4)

In [397]:
final_pred.shape

(44229, 1)

In [399]:
trainingset2_predicted = pd.concat([trainingset2,final_pred],axis=1)

In [400]:
trainingset2_predicted.shape

(44229, 5)

In [401]:
trainingset2_predicted.to_csv('trainingset2_predicted.csv')

In [404]:
pd.crosstab(trainingset2_predicted.leg,trainingset2_predicted.pred)

pred,0,1
leg,,
0,38647,7
1,0,5575


In [455]:
sam = 1

In [456]:
import pickle
Non_Leg = trainingset1[trainingset1['Label']== 0].sample(800,random_state=sam,replace=False)

Leg = trainingset1[trainingset1['Label']==1]
df_sampled = pd.concat([Non_Leg,Leg],axis=0)

X = df_sampled.text
y = df_sampled.Label



X_train_txt, X_test_txt, y_train, y_test,state_train,state_test = train_test_split(df_sampled.text,df_sampled.Label,
                                                                                   df_sampled.leg,test_size=0.2, random_state=1)
X_dtm_train = tfidf_vector.fit_transform((X_train_txt.values.astype('U')))
state_train = np.array(state_train)

dense = X_dtm_train.todense()
state_train = state_train.reshape(-1,1)
final_train_dtm = np.append(dense,state_train,1)

#print(X_dtm_train.shape)
X_dtm_test = tfidf_vector.transform(X_test_txt.values.astype('U'))
dense_test = X_dtm_test.todense()
state_test = np.array(state_test)
state_test = state_test.reshape(-1,1)
final_test_dtm = np.append(dense_test,state_test,1)
print(final_test_dtm.shape)
#svcclassifier = SVC(random_state=0,kernel='linear',C=3,probability=True)
#svcclassifier.fit(X_dtm_train,y_train)
#y_pred_svm = svcclassifier.predict(X_dtm_test)

rf_classifer = RandomForestClassifier(n_estimators=100,criterion='gini',random_state=0,class_weight={0:.3,1:.7})
rf_classifer.fit(final_train_dtm ,y_train)
y_pred_randforest = rf_classifer.predict(final_test_dtm)

print("RandForest ACCURACY:",metrics.accuracy_score(y_test,y_pred_randforest))
print('RandForest CLASSIFIER CONFUSTION MATRIX:',pd.crosstab(y_test,y_pred_randforest))
print("===> **RandForest F1 SCORE",metrics.f1_score(y_test, y_pred_randforest,average=None))
print("===> **RandForest Recall SCORE",metrics.recall_score(y_test, y_pred_randforest,average=None))
print("===> **RandForest Precision SCORE",metrics.precision_score(y_test, y_pred_randforest,average=None))
print("===> **RandForest ROC_AUC SCORE",metrics.roc_auc_score(y_test, y_pred_randforest,average=None))

#logreg = LogisticRegression(C=1e9)
#logreg.fit(X_dtm_train,y_train)
#y_pred_class_logreg = logreg.predict(X_dtm_test)
#rf_1 = 

#svcmodel = 'SVC'+'_'+str(sam)+'.sav'
#pickle.dump(svcclassifier,open(svcmodel,'wb'))

#rand = 'RF'+'_'+str(sam)+'.sav'
#pickle.dump(rf_classifer,open(rand,'wb'))

#log = 'Log'+'_'+str(sam)+'.sav'
#pickle.dump(logreg,open(log,'wb'))


(311, 3001)
RandForest ACCURACY: 0.7877813504823151
RandForest CLASSIFIER CONFUSTION MATRIX: col_0    0    1
Label          
0      128   39
1       27  117
===> **RandForest F1 SCORE [0.79503106 0.78      ]
===> **RandForest Recall SCORE [0.76646707 0.8125    ]
===> **RandForest Precision SCORE [0.82580645 0.75      ]
===> **RandForest ROC_AUC SCORE 0.7894835329341318


In [457]:
X.shape

(1555,)

In [65]:
vectorizer2 = 'vectorizer2.pk'
pickle.dump(tfidf_vector,open(vectorizer2,'wb'))

In [223]:
y_pred_proba_rf2 = rf_classifer.predict_proba(final_test_dtm)


In [171]:
y_pred_proba = pd.DataFrame(y_pred_proba_rf1[:,0],columns=['rf1'])

In [224]:
y_pred_proba_rf2 = pd.DataFrame(y_pred_proba_rf2[:,0],columns=['rf10'])

In [225]:
y_pred_proba = pd.concat([y_pred_proba,y_pred_proba_rf2],axis=1)

In [226]:
y_pred_proba.head()

,rf1,rf2,rf3,rf4,rf5,rf6,rf7,rf8,rf9,rf10
0,0.96,0.15,0.470000,0.97,0.97,0.91,0.270000,0.85,1.00,0.690000
1,0.87,0.93,0.870000,0.52,1.00,0.82,1.000000,0.75,0.19,0.940000
2,0.76,0.84,1.000000,0.56,0.67,0.86,0.990000,0.92,0.88,0.640000
3,0.24,0.29,0.464615,0.27,0.29,0.32,0.272432,0.30,0.32,0.344615
4,0.24,0.25,0.220000,0.27,0.25,0.19,0.240000,0.14,0.22,0.160000


In [227]:
y_test.shape

(311,)

In [228]:
y_p = y_test

In [229]:
X_prob = y_pred_proba

In [230]:
X_prob.shape

(311, 10)

ml = []
lab = []
for j in output[0]:
    for i in j:
        l.append(i[0])
for k in output[1]:
    for m in k:
        lab.append(m)
#(output[1][:,0]))

In [231]:
# split X and y into training and testing data sets
from sklearn.cross_validation import train_test_split

X_train_prob, X_test_prob, y_trainp, y_testp, = train_test_split(X_prob, y_p,test_size=0.2, random_state=1)

C:\Users\madhan.s\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [232]:
from sklearn.ensemble import RandomForestClassifier
rand_2nd_layer = RandomForestClassifier(n_estimators=200,criterion='entropy',random_state=0,class_weight={0:.4,1:0.6},max_depth=200)
rand_2nd_layer.fit(X_train_prob,y_trainp)
y_pred_2ndlayer = rand_2nd_layer.predict(X_test_prob)

In [233]:
X_train_prob.shape

(248, 10)

In [234]:
print("##################################################################################################")
print("RandForest ACCURACY:",metrics.accuracy_score(y_testp,y_pred_2ndlayer))
print('RandForest CLASSIFIER CONFUSTION MATRIX:',pd.crosstab(y_testp,y_pred_2ndlayer))
print("===> **RandForest F1 SCORE",metrics.f1_score(y_testp, y_pred_2ndlayer,average=None))
print("===> **RandForest Recall SCORE",metrics.recall_score(y_testp, y_pred_2ndlayer,average=None))
print("===> **RandForest Precision SCORE",metrics.precision_score(y_testp, y_pred_2ndlayer,average=None))
print("===> **RandForest ROC_AUC SCORE",metrics.roc_auc_score(y_testp, y_pred_2ndlayer,average=None))

##################################################################################################
RandForest ACCURACY: 1.0
RandForest CLASSIFIER CONFUSTION MATRIX: col_0   0   1
Label        
0      36   0
1       0  27
===> **RandForest F1 SCORE [1. 1.]
===> **RandForest Recall SCORE [1. 1.]
===> **RandForest Precision SCORE [1. 1.]
===> **RandForest ROC_AUC SCORE 1.0


In [235]:
rand_2 = 'RF_2nd_layer.sav'
pickle.dump(rand_2nd_layer,open(rand_2,'wb'))

In [236]:
test_path = r'C:\Madhan\Analytics\Machine_Learning_project_work\AI_ML\bluepencil\model_improvement'
pymu_test = pd.read_excel(test_path+'\Label1_Pymupdf_TokenizedSentences.xlsx',sheet_name='Sheet1')
bptool_files = pd.read_csv(r'C:\Madhan\Analytics\Machine_Learning_project_work\AI_ML\bluepencil\model_improvement\bp_tool_files_test.csv')

In [237]:
bptool_files.head()

,Index,filename,text,keyword
0,0,1004168_D7514_HRT Placemat_111816_vFINAL - Cop...,The Next HR Transformation The Business Led HR...,No
1,1,1004168_D7514_HRT Placemat_111816_vFINAL - Cop...,HR strategist risk advisor talent manager Virt...,yes
2,2,1004168_D7514_HRT Placemat_111816_vFINAL - Cop...,All Rights Reserved.,No
3,3,1004168_D7514_HRT Placemat_111816_vFINAL - Cop...,ED None.,No
4,4,1004168_D7514_HRT Placemat_111816_vFINAL - Cop...,EYG no.,No


In [238]:
bptool_files['leg'] = ''
leg = []
for idx, row in bptool_files.iterrows():
    line_txt = bptool_files.text[idx]
    #print(line_txt)
    for row1 in LGL_words.LGL:
        #print(row1)
        if row1 in line_txt:
            bptool_files['leg'][idx] = 'yes'

C:\Users\madhan.s\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [239]:
bptool_files.leg.value_counts()

       727
yes    188
Name: leg, dtype: int64

In [240]:
bptool_files.leg.replace(['','yes'],[0,1],inplace=True)

In [241]:
bptool_files.tail()

,Index,filename,text,keyword,leg
910,910,Why a robust production system is a prerequisi...,Over time employee internalize habit important...,No,0
911,911,Why a robust production system is a prerequisi...,When enough people develop behavior affect lar...,No,0
912,912,Why a robust production system is a prerequisi...,That result company seek become sustainable fo...,No,0
913,913,Why a robust production system is a prerequisi...,A production system eliminates traditional dis...,No,0
914,914,Why a robust production system is a prerequisi...,Done right make continually improve business s...,No,0


In [242]:
X_test_pymu = bptool_files.text

In [243]:
X_test_pymu.shape

(915,)

In [244]:
print("Number of Rows Parsed......................\n")
corpus = []
num_SD = len(X_test_pymu)
%time
for i in range( 0, num_SD ):
    if( (i+1)%5000 == 0 ):
    # If the index is evenly divisible by 1000, print a message
        print("Cleaned Short Descriptions %d of %d" % ( i+1, num_SD ))
    corpus.append( review_to_words(str(X_test_pymu.iloc[i])))

Number of Rows Parsed......................

Wall time: 0 ns


In [245]:
X_test_pymu.iloc[1]

'HR strategist risk advisor talent manager Virtual team align HR business leader address problem develop solution Innovation center share service design improve quality speed decrease cost Integrated HR business predictive analytics drive business strategy organizational decision make Globally consistent yet locally sensitive program policy procedure ensure compliance people engagement Scalable preference base mobile social system technology tool facilitate service delivery networking employee engagement framework guide transformation decision make EYGM Limited.'

In [246]:
corpus[1]

'strategist risk advisor talent manager virtual team align business leader address problem develop solution innovation center share service design improve quality speed decrease cost integrate business predictive analytics drive business strategy organizational decision make globally consistent yet locally sensitive program policy procedure ensure compliance people engagement scalable preference base mobile social system technology tool facilitate service delivery networking employee engagement framework guide transformation decision make eygm limited'

In [279]:
X_testcleaned = pd.DataFrame(corpus)
X_dtm_test = tfidf_vector.transform(X_testcleaned[0])

In [280]:
X_dtm_test.shape

(915, 3000)

In [281]:
bptool_files.leg.shape

(915,)

In [282]:
dense_test = X_dtm_test.todense()
state_test = np.array(bptool_files.leg)
state_test = state_test.reshape(-1,1)
final_test_dtm = np.append(dense_test,state_test,1)

In [283]:
final_test_dtm.shape

(915, 3001)

In [284]:
rf_1 = pickle.load(open(test_path+'\RF_1.sav', 'rb'))
rf_2 = pickle.load(open(test_path+'\RF_2.sav', 'rb'))
rf_3 = pickle.load(open(test_path+'\RF_3.sav', 'rb'))
rf_4 = pickle.load(open(test_path+'\RF_4.sav', 'rb'))
rf_5 = pickle.load(open(test_path+'\RF_5.sav', 'rb'))
rf_6 = pickle.load(open(test_path+'\RF_6.sav', 'rb'))
rf_7 = pickle.load(open(test_path+'\RF_7.sav', 'rb'))
rf_8 = pickle.load(open(test_path+'\RF_8.sav', 'rb'))
rf_9 = pickle.load(open(test_path+'\RF_9.sav', 'rb'))
rf_10 = pickle.load(open(test_path+'\RF_10.sav', 'rb'))

In [285]:
y_pred_model1 = rf_1.predict_proba(final_test_dtm)
y_pred_model2 = rf_2.predict_proba(final_test_dtm)
y_pred_model3 = rf_3.predict_proba(final_test_dtm)
y_pred_model4 = rf_4.predict_proba(final_test_dtm)
y_pred_model5 = rf_5.predict_proba(final_test_dtm)
y_pred_model6 = rf_6.predict_proba(final_test_dtm)
y_pred_model7 = rf_7.predict_proba(final_test_dtm)
y_pred_model8 = rf_8.predict_proba(final_test_dtm)
y_pred_model9 = rf_9.predict_proba(final_test_dtm)
y_pred_model10 = rf_10.predict_proba(final_test_dtm)

In [286]:
n=np.vstack((y_pred_model1[:,0],y_pred_model2[:,0],y_pred_model3[:,0],y_pred_model4[:,0],y_pred_model5[:,0],
             y_pred_model6[:,0],y_pred_model7[:,0],
            y_pred_model8[:,0],y_pred_model9[:,0],y_pred_model10[:,0]))

In [287]:
n.shape

(10, 915)

In [288]:
n[0:,]

array([[0.86      , 0.33      , 0.98      , ..., 0.94      , 1.        ,
        0.873     ],
       [0.84024038, 0.30194079, 0.99      , ..., 0.92      , 0.85      ,
        0.86      ],
       [0.56      , 0.17593117, 0.99      , ..., 0.82      , 0.64      ,
        0.58      ],
       ...,
       [0.83      , 0.38      , 1.        , ..., 0.97      , 0.87      ,
        0.95      ],
       [0.9       , 0.43631579, 0.94      , ..., 0.95      , 0.94517241,
        0.92      ],
       [0.38      , 0.13461538, 1.        , ..., 0.81      , 0.68      ,
        0.58      ]])

In [289]:
n = n.transpose()

In [290]:
final_df = n

In [291]:
rf_2nd = pickle.load(open(test_path+'\RF_2nd_layer.sav', 'rb'))

In [292]:
final_pred = rf_2nd.predict(final_df)

In [293]:
final_pred

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,

In [294]:
final_pred = pd.DataFrame(final_pred,columns=['pred'])

In [295]:
X_testcleaned = pd.concat([X_testcleaned,final_pred,bptool_files.filename],axis=1)

In [297]:
X_testcleaned.filename.shape

(915,)

In [298]:
X_testcleaned.shape

(915, 3)

In [300]:
#X_testcleaned.iloc[6321]

In [301]:
X_testcleaned.to_csv(r'C:\Madhan\Analytics\Machine_Learning_project_work\AI_ML\bluepencil\model_improvement\bptoolfiles_predicted.csv')

## Neural Network Model

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
from keras.datasets import imdb
import numpy as np
from keras.layers.normalization import BatchNormalization

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers import LSTM
from keras import callbacks

In [6]:
trainingset2.head()

,Index,filename,text,leg
0,0,-01-0290_Folder Tax_Vat_2017_Internet.pdf,Please inform principle reciprocity rule requi...,0
1,1,-01-0290_Folder Tax_Vat_2017_Internet.pdf,June follow year September follow year June fo...,1
2,2,-01-0290_Folder Tax_Vat_2017_Internet.pdf,In particular Asian company must Not residence...,1
3,3,-01-0290_Folder Tax_Vat_2017_Internet.pdf,Claims must also comply certain specific admin...,0
4,4,-01-0290_Folder Tax_Vat_2017_Internet.pdf,Some EU country also require company country r...,0


In [11]:
trainingset1.shape

(30801, 4)

In [133]:
X = trainingset1.text
y = trainingset1.Label

In [134]:
sam = 3

In [135]:
Non_Leg = trainingset1[trainingset1['Label']== 0].sample(800,random_state=sam,replace=False)

Leg = trainingset1[trainingset1['Label']==1]
df_sampled = pd.concat([Non_Leg,Leg],axis=0)

X = df_sampled.text
y = df_sampled.Label



X_train_txt, X_test_txt, y_train, y_test,state_train,state_test = train_test_split(df_sampled.text,df_sampled.Label,
                                                                                   df_sampled.leg,test_size=0.2, random_state=1)

In [136]:
X.shape

(1555,)

In [30]:
X_train_txt, X_test_txt, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=1)

In [31]:
print(X_train_txt.shape)
print(X_test_txt.shape)

(1244,)
(311,)


In [32]:
max_words = 3000
max_len = 300
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train_txt.values.astype('U'))
sequences = tok.texts_to_sequences(X_train_txt.values.astype('U'))
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [33]:
sequences_matrix.shape

(1244, 300)

In [34]:
sequences_test = tok.texts_to_sequences(X_test_txt.values.astype('U'))
sequences_matrix_test = sequence.pad_sequences(sequences_test,maxlen=max_len)

In [35]:
print(sequences_matrix[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [36]:
word_id = tok.word_index

In [37]:
INDEX_FROM = 3

word_id = {k:(v+INDEX_FROM) for k,v in word_id.items()} # Shift all the items to make space for the SPECIAL WORDS
word_id["_UNK_"] = 0
word_id["_START_"] = 1
word_id["_CUT_"] = 2

In [38]:
id_word = {v:k for k,v in word_id.items()}

In [39]:
id_word[2]

'_CUT_'

In [40]:
sequences_matrix[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [41]:
print(' '.join(id_word[i] for i in sequences_matrix[1] ))

_UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK_ _UNK

In [42]:
vanilla_model = Sequential()
vanilla_model.add(Embedding(3000, 500, input_length=max_len)) #vOCABSIZE IS 5000
#vanilla_model.add(BatchNormalization())
vanilla_model.add(Flatten())
vanilla_model.add(Dense(250, activation='relu'))
vanilla_model.add(Dropout(0.2))
vanilla_model.add(Dense(1, activation='sigmoid'))
vanilla_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

NameError: name 'Dropout' is not defined

In [132]:
vanilla_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 300, 500)          1500000   
_________________________________________________________________
flatten_13 (Flatten)         (None, 150000)            0         
_________________________________________________________________
dense_25 (Dense)             (None, 250)               37500250  
_________________________________________________________________
dropout_13 (Dropout)         (None, 250)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 251       
Total params: 39,000,501
Trainable params: 39,000,501
Non-trainable params: 0
_________________________________________________________________


In [133]:
vanilla_model.fit(sequences_matrix, y_train, batch_size=64, epochs=50,
          validation_data=(sequences_matrix_test, y_test),verbose=2)

Train on 1244 samples, validate on 311 samples
Epoch 1/50
 - 29s - loss: 1.4237 - acc: 0.5675 - val_loss: 0.5827 - val_acc: 0.6881
Epoch 2/50
 - 25s - loss: 0.5083 - acc: 0.7540 - val_loss: 0.5110 - val_acc: 0.7395
Epoch 3/50
 - 28s - loss: 0.2810 - acc: 0.9092 - val_loss: 0.4783 - val_acc: 0.7942
Epoch 4/50
 - 28s - loss: 0.1380 - acc: 0.9654 - val_loss: 0.4520 - val_acc: 0.8199
Epoch 5/50
 - 25s - loss: 0.0671 - acc: 0.9871 - val_loss: 0.4698 - val_acc: 0.8167
Epoch 6/50
 - 26s - loss: 0.0416 - acc: 0.9920 - val_loss: 0.4766 - val_acc: 0.8103
Epoch 7/50
 - 26s - loss: 0.0317 - acc: 0.9912 - val_loss: 0.4825 - val_acc: 0.8199
Epoch 8/50
 - 26s - loss: 0.0269 - acc: 0.9920 - val_loss: 0.5268 - val_acc: 0.7846
Epoch 9/50
 - 28s - loss: 0.0213 - acc: 0.9928 - val_loss: 0.5038 - val_acc: 0.8071
Epoch 10/50
 - 25s - loss: 0.0217 - acc: 0.9928 - val_loss: 0.5283 - val_acc: 0.8232
Epoch 11/50
 - 26s - loss: 0.0204 - acc: 0.9928 - val_loss: 0.5222 - val_acc: 0.8103
Epoch 12/50
 - 25s - loss: 

In [127]:
pred_NN = vanilla_model.predict_classes(sequences_matrix_test)

In [128]:
pred_NN = pd.DataFrame(pred_NN)

In [129]:
metrics.confusion_matrix(y_test,pred_NN)

array([[151,  16],
       [ 47,  97]], dtype=int64)

In [130]:
print("NN ACCURACY:",metrics.accuracy_score(y_test,pred_NN))
print('NN CLASSIFIER CONFUSTION MATRIX:',metrics.confusion_matrix(y_test,pred_NN))
print("===> **NN F1 SCORE",metrics.f1_score(y_test, pred_NN,average=None))
print("===> **NN Recall SCORE",metrics.recall_score(y_test, pred_NN,average=None))
print("===> **NN Precision SCORE",metrics.precision_score(y_test, pred_NN,average=None))
print("===> **NN ROC_AUC SCORE",metrics.roc_auc_score(y_test, pred_NN,average=None))
print("#######################################################################################")

NN ACCURACY: 0.797427652733119
NN CLASSIFIER CONFUSTION MATRIX: [[151  16]
 [ 47  97]]
===> **NN F1 SCORE [0.82739726 0.75486381]
===> **NN Recall SCORE [0.90419162 0.67361111]
===> **NN Precision SCORE [0.76262626 0.85840708]
===> **NN ROC_AUC SCORE 0.7889013639387891
#######################################################################################


## LSTM Network

In [25]:
from keras.datasets import imdb
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers import LSTM
from keras import callbacks
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence


Using TensorFlow backend.


In [46]:
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000 #vocab size
maxlen = 300  #number of word used from each review
tok = Tokenizer(num_words=vocab_size)
tok.fit_on_texts(X_train_txt.astype('U'))
sequences_train = tok.texts_to_sequences(X_train_txt.astype('U'))
sequences_test = tok.texts_to_sequences(X_test_txt)
#load dataset as a list of ints
#(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
#make all sequences of the same length
x_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)
x_test =  sequence.pad_sequences(sequences_test, maxlen=maxlen)

In [96]:
lstm_model = Sequential()
lstm_model.add(Embedding(vocab_size, 128))
lstm_model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
#lstm_model.add(
lstm_model.add(Dense(100, activation='relu'))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(1, activation='sigmoid'))

early_stopping = callbacks.EarlyStopping(monitor='val_loss', min_delta=0,
                                         patience=2, verbose=0, mode='auto')

# Compile and train.
lstm_model.compile(loss='binary_crossentropy', 
              optimizer='sgd',
              metrics=['accuracy'])

In [100]:
lstm_model.fit(x_train, y_train, batch_size=64, epochs=2,
          validation_data=(x_test, y_test),
          callbacks=[early_stopping])

loss, acc = lstm_model.evaluate(x_test, y_test, batch_size=64)

print('Test loss (LOWER is better)      :', loss)
print('Test accuracy (HIGHER is better) :', acc)

Train on 24640 samples, validate on 6161 samples
Epoch 1/2
12992/24640 [==============>...............] - ETA: 29:03 - loss: 0.5374 - acc: 0.98 - ETA: 30:28 - loss: 0.5411 - acc: 0.96 - ETA: 30:12 - loss: 0.5362 - acc: 0.97 - ETA: 30:08 - loss: 0.5326 - acc: 0.98 - ETA: 29:53 - loss: 0.5303 - acc: 0.98 - ETA: 29:33 - loss: 0.5268 - acc: 0.98 - ETA: 29:40 - loss: 0.5259 - acc: 0.98 - ETA: 29:59 - loss: 0.5244 - acc: 0.98 - ETA: 30:04 - loss: 0.5227 - acc: 0.98 - ETA: 30:05 - loss: 0.5217 - acc: 0.98 - ETA: 30:03 - loss: 0.5204 - acc: 0.98 - ETA: 29:58 - loss: 0.5193 - acc: 0.98 - ETA: 29:43 - loss: 0.5168 - acc: 0.98 - ETA: 29:43 - loss: 0.5159 - acc: 0.97 - ETA: 29:39 - loss: 0.5136 - acc: 0.98 - ETA: 29:35 - loss: 0.5122 - acc: 0.98 - ETA: 29:32 - loss: 0.5099 - acc: 0.98 - ETA: 29:29 - loss: 0.5086 - acc: 0.98 - ETA: 29:27 - loss: 0.5063 - acc: 0.98 - ETA: 29:29 - loss: 0.5043 - acc: 0.98 - ETA: 29:24 - loss: 0.5026 - acc: 0.98 - ETA: 29:20 - loss: 0.5013 - acc: 0.98 - ETA: 29:18 - l

13056/24640 [==============>...............] - ETA: 36:19 - loss: 0.2425 - acc: 0.93 - ETA: 36:04 - loss: 0.1863 - acc: 0.95 - ETA: 35:50 - loss: 0.1730 - acc: 0.95 - ETA: 35:47 - loss: 0.1667 - acc: 0.96 - ETA: 35:47 - loss: 0.1817 - acc: 0.95 - ETA: 35:47 - loss: 0.1675 - acc: 0.96 - ETA: 35:41 - loss: 0.1640 - acc: 0.96 - ETA: 35:36 - loss: 0.1559 - acc: 0.96 - ETA: 35:36 - loss: 0.1582 - acc: 0.96 - ETA: 35:33 - loss: 0.1567 - acc: 0.96 - ETA: 35:29 - loss: 0.1597 - acc: 0.96 - ETA: 35:08 - loss: 0.1537 - acc: 0.96 - ETA: 35:02 - loss: 0.1492 - acc: 0.96 - ETA: 34:53 - loss: 0.1450 - acc: 0.96 - ETA: 34:47 - loss: 0.1417 - acc: 0.96 - ETA: 34:44 - loss: 0.1390 - acc: 0.96 - ETA: 34:48 - loss: 0.1421 - acc: 0.96 - ETA: 34:50 - loss: 0.1422 - acc: 0.96 - ETA: 34:45 - loss: 0.1393 - acc: 0.96 - ETA: 34:39 - loss: 0.1365 - acc: 0.97 - ETA: 34:23 - loss: 0.1364 - acc: 0.97 - ETA: 34:14 - loss: 0.1345 - acc: 0.97 - ETA: 34:06 - loss: 0.1307 - acc: 0.97 - ETA: 34:02 - loss: 0.1307 - acc: 

6161/6161 [==============================] - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 59s - ETA: 58 - ETA: 57 - ETA: 56 - ETA: 55 - ETA: 55 - ETA: 54 - ETA: 54 - ETA: 53 - ETA: 52 - ETA: 52 - ETA: 51 - ETA: 50 - ETA: 49 - ETA: 48 - ETA: 47 - ETA: 46 - ETA: 45 - ETA: 44 - ETA: 44 - ETA: 43 - ETA: 42 - ETA: 41 - ETA: 40 - ETA: 40 - ETA: 39 - ETA: 38 - ETA: 37 - ETA: 36 - ETA: 35 - ETA: 34 - ETA: 33 - ETA: 32 - ETA: 31 - ETA: 31 - ETA: 30 - ETA: 29 - ETA: 28 - ETA: 27 - ETA: 26 - ETA: 25 - ETA: 24 - ETA: 24 - ETA: 23 - ETA: 22 - ETA: 21 - ETA: 20 - ETA: 19 - ETA: 18 - ETA: 18 - ETA: 17 - ETA: 16 - ETA: 15 - ETA: 14 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 81s 13ms/step
Tes

In [101]:
lstm_pred = lstm_model.predict_classes(sequences_matrix_test)

In [102]:
print("LSTM ACCURACY:",metrics.accuracy_score(y_test,lstm_pred))
print('LSTM CLASSIFIER CONFUSTION MATRIX:',metrics.confusion_matrix(y_test,lstm_pred))
print("===> **LSTM F1 SCORE",metrics.f1_score(y_test, lstm_pred,average=None))
print("===> **LSTM Recall SCORE",metrics.recall_score(y_test, lstm_pred,average=None))
print("===> **LSTM Precision SCORE",metrics.precision_score(y_test, lstm_pred,average=None))
print("===> **LSTM ROC_AUC SCORE",metrics.roc_auc_score(y_test, lstm_pred,average=None))
print("#######################################################################################")

LSTM ACCURACY: 0.9766271709138127
LSTM CLASSIFIER CONFUSTION MATRIX: [[6017    0]
 [ 144    0]]


C:\Users\madhan.s\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


===> **LSTM F1 SCORE [0.9881754 0.       ]
===> **LSTM Recall SCORE [1. 0.]


C:\Users\madhan.s\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


===> **LSTM Precision SCORE [0.97662717 0.        ]
===> **LSTM ROC_AUC SCORE 0.5
#######################################################################################


In [374]:
test = pd.read_csv(r'C:\Madhan\Analytics\Machine_Learning_project_work\AI_ML\bluepencil\model_improvement\test_set_1.csv')

In [375]:
test.head()

,Unnamed: 0,filename,fileContent,Label
0,0,-01-0290_Folder Tax_Vat_2017_Internet.pdf,13th Directive \r\r\nEU VAT refund \r\r\nreque...,Legal
1,1,-01-0290_Folder Tax_Vat_2017_Internet.pdf,Country\r\r\nAustria \r\r\nBelgium\r\r\nBulgar...,Non-Legal
2,2,-01-0290_Folder Tax_Vat_2017_Internet.pdf,2\r\r\n\r\r\n| 13th Directive EU VAT refund ...,Legal
3,3,-01-0290_Folder Tax_Vat_2017_Internet.pdf,"A company established in Hong Kong, Taiwan, Ma...",Non-Legal
4,4,-01-0290_Folder Tax_Vat_2017_Internet.pdf,"In particular, \r\r\nthe Asian company must:\r...",Non-Legal


In [377]:
X = test.fileContent

In [378]:
print("Number of Rows Parsed......................\n")
corpus = []
num_SD = len(X)
%time
for i in range( 0, num_SD ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%5000 == 0 ):
        print("Cleaned Short Descriptions %d of %d" % ( i+1, num_SD ))
    corpus.append( review_to_words(str(X.iloc[i])))

Number of Rows Parsed......................

Wall time: 0 ns
Cleaned Short Descriptions 5000 of 39055
Cleaned Short Descriptions 10000 of 39055
Cleaned Short Descriptions 15000 of 39055
Cleaned Short Descriptions 20000 of 39055
Cleaned Short Descriptions 25000 of 39055
Cleaned Short Descriptions 30000 of 39055
Cleaned Short Descriptions 35000 of 39055


In [379]:
X_dtm_test = tfidf_vec.transform(corpus)

In [380]:
X_dtm_test.shape

(39055, 255)

In [381]:
y_pred_randforest = rand_classifer.predict(X_dtm_test)

In [384]:
y_pred_randforest = pd.DataFrame(y_pred_randforest)

In [385]:
test_1 = pd.concat([test,y_pred_randforest],axis=1)

In [386]:
test_1.to_csv('test1.csv')

## BERT Embedding

In [127]:
from bert_embedding import BertEmbedding

In [128]:
bert = BertEmbedding()

In [137]:
X.head()

266      despite instance runaway success rather compet...
2905          thing people work tax tend rely gut instinct
9425     simplification investment bank tangle estate a...
29280    majority key fund structure decision make basi...
15696    firm across industry already take step prepare...
Name: text, dtype: object

In [458]:
y.shape

(1555,)

In [138]:
sentences = X

In [146]:
sentences =list(sentences)

In [190]:
sentences[0:4]

['despite instance runaway success rather compete directly see fintechs bank exist symbiotically create strong financial system',
 'thing people work tax tend rely gut instinct',
 'simplification investment bank tangle estate arguably difficult cost address also one struggle tackle',
 'majority key fund structure decision make basis two critical commercial consideration directly affect tax position would apply fund investor fund management team track record fund management team type geographic location prospective investor']

def corpus_vector(word2vec_model, corpus):
    new_corpus=[]
    for doc in corpus:
        new_doc = []
        for i in range(0,50):
            if i < len(doc) and doc[i] in word2vec_model.vocab:
                new_doc.append(word2vec_model[doc[i]])
            else:
                new_doc.append([0]*300)
        new_corpus.append(new_doc)
    # remove out-of-vocabulary words
    #doc = [word for word in doc if word in word2vec_model.vocab]
    #print(doc)
    return np.array(new_corpus)

In [398]:
from datetime import datetime

In [400]:
print(datetime.now())

2019-07-29 13:59:31.881639


In [529]:
array = np.zeros((1555,50,768))

In [510]:
array.shape

(1555, 50, 768)

In [528]:
print("Number of Rows Parsed......................\n")
bert_embed = []
num_SD = len(sentences)
print(datetime.now())
for i in range( 0, num_SD):
    doc = nltk.sent_tokenize(sentences[i])
    #print(doc)
    result = bert_embedding(doc,oov_way='avg')
    #print(result)
    bert_array = np.array(result[0][1])
        #print(bert_array.shape)
    bert_embed.append(bert_array)
    np.append(array[i],bert_array)
        #print(len(bert_embed))
    if( (i+1)%250 == 0 ):
        print("Cleaned Short Descriptions %d of %d" % ( i+1, num_SD ))
        print(i)
        print(datetime.now())

Number of Rows Parsed......................

2019-07-30 11:38:51.935663


KeyboardInterrupt: 

In [491]:
array

array([[ 0.9343181 ,  0.3573656 ,  0.25016475, ..., -0.36225465,
         0.49115077,  0.01891416],
       [ 0.57757705, -0.11281667,  0.35245678, ..., -0.19790722,
         0.26295584,  0.07346324],
       [ 0.72832364,  0.30656624,  0.6930689 , ..., -0.30307057,
         0.13134867, -0.08489612],
       ...,
       [-0.19381317,  0.26194164,  0.2444029 , ..., -0.20700751,
        -0.31119695, -0.9367102 ],
       [-0.7129689 , -0.07343789,  0.5752077 , ..., -0.13918841,
        -0.34515035,  0.4092333 ],
       [-0.09110375,  0.35341305,  0.12333003, ...,  0.25723186,
        -0.32090333, -0.08453374]], dtype=float32)

In [531]:
arr = []
for b in bert_embed[0:5]:
    print(b.shape)
    arr.append(b)
    #np.append(array,b)

(16, 768)
(8, 768)
(13, 768)
(23, 768)
(15, 768)


In [559]:
emb = []
for a in arr[0]:
    emb.append(a)

In [560]:
c = np.array(emb)

In [567]:
c.shape

(16, 768)

In [565]:
d.shape

(59724288,)

In [555]:
d.shape

(59724288,)

In [540]:
len(emb)

8

In [522]:
array.shape

(59769600,)

In [489]:
arr[0].shape

(16, 768)

In [473]:
bert_embed = np.array(bert_embed)

In [474]:
bert_embed.shape

(1555,)

In [477]:
bert_embed[1].shape

(8, 768)

In [478]:
bert_embed.shape

(1555,)

In [459]:
bert_embed.shape

(1555, 1, 768)

In [460]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [461]:
def RNN():
    inputs = Input(name='inputs',shape=[1,768])
    #layer = Embedding(1000,50,input_length=max_len)(inputs)
    layer = LSTM(64)(inputs)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.3)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [462]:
max_len = [1,768]

In [463]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1, 768)            0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 64)                213248    
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_17 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
_________________________________________________________________
activation_18 (Activation)   (None, 1)                 0         
Total para

In [467]:
model.fit(bert_embed,y,batch_size=32,epochs=50,
          validation_split=0.2)

Train on 1244 samples, validate on 311 samples
Epoch 1/50
1244/1244 [==============================] - ETA: 0s - loss: 0.6581 - acc: 0.687 - ETA: 0s - loss: 0.4903 - acc: 0.770 - ETA: 0s - loss: 0.4719 - acc: 0.771 - ETA: 0s - loss: 0.4796 - acc: 0.755 - ETA: 0s - loss: 0.4796 - acc: 0.753 - ETA: 0s - loss: 0.4834 - acc: 0.752 - ETA: 0s - loss: 0.4832 - acc: 0.759 - ETA: 0s - loss: 0.4848 - acc: 0.763 - 0s 384us/step - loss: 0.4818 - acc: 0.7629 - val_loss: 1.4059 - val_acc: 0.2605
Epoch 2/50
1244/1244 [==============================] - ETA: 0s - loss: 0.4784 - acc: 0.750 - ETA: 0s - loss: 0.4569 - acc: 0.775 - ETA: 0s - loss: 0.4386 - acc: 0.795 - ETA: 0s - loss: 0.4391 - acc: 0.796 - ETA: 0s - loss: 0.4222 - acc: 0.809 - ETA: 0s - loss: 0.4242 - acc: 0.807 - ETA: 0s - loss: 0.4302 - acc: 0.803 - ETA: 0s - loss: 0.4379 - acc: 0.799 - ETA: 0s - loss: 0.4434 - acc: 0.797 - 1s 433us/step - loss: 0.4512 - acc: 0.7934 - val_loss: 1.5943 - val_acc: 0.1833
Epoch 3/50
1244/1244 [=============

1244/1244 [==============================] - ETA: 0s - loss: 0.1884 - acc: 0.906 - ETA: 0s - loss: 0.2759 - acc: 0.833 - ETA: 0s - loss: 0.2935 - acc: 0.822 - ETA: 0s - loss: 0.2800 - acc: 0.828 - ETA: 0s - loss: 0.2879 - acc: 0.831 - ETA: 0s - loss: 0.2837 - acc: 0.835 - ETA: 0s - loss: 0.2883 - acc: 0.835 - ETA: 0s - loss: 0.2902 - acc: 0.832 - 0s 377us/step - loss: 0.2935 - acc: 0.8336 - val_loss: 2.2678 - val_acc: 0.3119
Epoch 37/50
1244/1244 [==============================] - ETA: 0s - loss: 0.2731 - acc: 0.843 - ETA: 0s - loss: 0.2550 - acc: 0.843 - ETA: 0s - loss: 0.2249 - acc: 0.868 - ETA: 0s - loss: 0.2551 - acc: 0.854 - ETA: 0s - loss: 0.2725 - acc: 0.843 - ETA: 0s - loss: 0.2752 - acc: 0.843 - ETA: 0s - loss: 0.2788 - acc: 0.842 - ETA: 0s - loss: 0.2855 - acc: 0.841 - ETA: 0s - loss: 0.2865 - acc: 0.838 - 0s 400us/step - loss: 0.2907 - acc: 0.8368 - val_loss: 2.8663 - val_acc: 0.3601
Epoch 38/50
1244/1244 [==============================] - ETA: 0s - loss: 0.2822 - acc: 0.843